# ICESat-2 Snow Depth Analyses: Land Cover and Slope

Other notebooks in this repository may be used to derive snow depth from ICESat-2 products. If the main notebook (is2_snowex_ak.ipynb) is used, then a user may generate a CSV of coregistered ICESat-2 and airborne lidar (UAF) surface heights and depths.

In this notebook, we are going to use pre-saved CSVs derived from said notebook to perform a land cover and slope analysis. The "ground_data_processing.py" and "lidar_processing.py" scripts have a few useful functions to help us with this.

The "xrspatial" package is also required for the slope analysis. It can be installed using the command "conda install -c conda-forge xarray-spatial".

In [ ]:
import ground_data_processing as gdp
import holoviews as hv
import hvplot.pandas
import numpy as np
import os
import pandas as pd
import rioxarray as rio
from xrspatial import hillshade, slope

hv.extension('bokeh')

Let's first load the presaved data for the available field sites in Alaska. The land cover data co-registration can be time consuming, so this notebook only matches with one site at a time, as defined by the user.

In [ ]:
# Load presaved data into DataFrames
acp_file = '/home/jovyan/icesat2-snowex/snow-depth-data/acp/atl03_snowdepth_rgt1097_acp_202234.csv'
bcef_file = '/home/jovyan/icesat2-snowex/snow-depth-data/bcef/atl03_snowdepth_rgt472_bcef_2022423.csv'
cffl_file = '/home/jovyan/icesat2-snowex/snow-depth-data/cffl/atl03_snowdepth_rgt1356_cffl_2022321.csv'
cpcrw_file = '/home/jovyan/icesat2-snowex/snow-depth-data/cpcrw/atl03_snowdepth_rgt1356_cpcrw_2022321.csv'

acp = pd.read_csv(acp_file, header=0)
bcef = pd.read_csv(bcef_file, header=0)
cffl = pd.read_csv(cffl_file, header=0)
cpcrw = pd.read_csv(cpcrw_file, header=0)

In [ ]:
# Field ID (same scheme as main notebook)
field_id = 'cffl'

if field_id == 'acp':
    # Derive time string from data
    acp['time'] = pd.to_datetime(acp['time'])
    acp = acp.set_index('time')

    time_str = str(acp.index.year[0]) + \
               str(acp.index.month[0]) + \
               str(acp.index.day[0])
    
    # Apply relevant ICESat-2 RGT (more generic scheme will be made in the future)
    rgt = '1097'
elif field_id == 'bcef':
    # Derive time string from data
    bcef['time'] = pd.to_datetime(bcef['time'])
    bcef = bcef.set_index('time')

    time_str = str(bcef.index.year[0]) + \
               str(bcef.index.month[0]) + \
               str(bcef.index.day[0])
    
    # Apply relevant ICESat-2 RGT (more generic scheme will be made in the future)
    rgt = '472'
elif field_id == 'cffl':
    # Derive time string from data
    cffl['time'] = pd.to_datetime(cffl['time'])
    cffl = cffl.set_index('time')

    time_str = str(cffl.index.year[0]) + \
               str(cffl.index.month[0]) + \
               str(cffl.index.day[0])
    
    # Apply relevant ICESat-2 RGT (more generic scheme will be made in the future)
    rgt = '1356'
elif field_id == 'cpcrw':
    # Derive time string from data
    cpcrw['time'] = pd.to_datetime(cpcrw['time'])
    cpcrw = cpcrw.set_index('time')

    time_str = str(cpcrw.index.year[0]) + \
               str(cpcrw.index.month[0]) + \
               str(cpcrw.index.day[0])
    
    # Apply relevant ICESat-2 RGT (more generic scheme will be made in the future)
    rgt = '1356'

## Match with Land Cover Data

The land cover analysis will use data from the National Land Cover Database (NLCD). The relevant land cover rasters over Alaska are in the "jsons-shps/" repository.

In [ ]:
# Read the NCLD raster
if (field_id=='cffl') | (field_id=='bcef') | (field_id=='cpcrw'):
    tiff = '/home/jovyan/icesat2-snowex/jsons-shps/land-cover-maps/fairbanks_nlcd_2016.tif'
elif (field_id=='acp') | (field_id=='trs'):
    tiff = '/home/jovyan/icesat2-snowex/jsons-shps/land-cover-maps/acp_nlcd_2016.tif'
else:
    print('No NLCD tiff found.')
    
land_cover = rio.open_rasterio(tiff)

land_cover

NLCD land cover types are given a string-based label and a numeric identifier. We want to preserve these exact numbers, so we cannot use the spline-based interpolation scheme we used for the lidar. So, the below cell instead uses a slower nearest-neighbor approach and saves coregistered land types into a text file.

Because ICESat-2 tracks vary slightly with each repeat cycle, the NLCD coregistration scheme will need to be run with any new ICESat-2 track. Otherwise, if the land cover text file already exists, then the below cell simply loads that text file.

In [ ]:
# Check if land cover data already exists for the field site
fname = f'coregistered_nlcd_rgt{rgt}_{field_id}_{time_str}.txt'
if os.path.isfile(fname):
    print('Land cover data already exists.')
else:
    gdp.coregister_nlcd_data(atl03_uaf_filtered, land_cover, fname)
    
atl03_uaf_nlcd = gdp.process_nlcd_data(cffl, fname)

print(np.unique(atl03_uaf_nlcd['land_cover']))

Let's now see how ICESat-2 snow depths are distributed across land cover type.

In [ ]:
# Make a boxplot grouping ICESat-2 snow depths by land cover
boxplot = atl03_uaf_nlcd.hvplot.box(y='residual', by='land_cover').opts(xlabel='Land Cover',
                                                                        ylabel='ICESat-2 snow depth [m]')
boxplot

## Slope Analysis

We are now going to perform a similar analysis related to slope patterns across the field sites. Note that the presaved data already includes derived slope estimates from the ICESat-2 ATL03 data. Just to make sure they are similar, we are going to estimate surface slope from the UAF lidar data as well.

In [ ]:
# Estimate along-track UAF slope
dh = cpcrw['lidar_height'].diff()
cpcrw['lidar_slope'] = np.arctan(dh/10) * (180/np.pi) # degrees

cpcrw['lidar_slope'].head()

In [ ]:
# Convert ICESat-2 slope to degrees
cpcrw['is2_slope'] = cpcrw['is2_slope'] * (180/np.pi)

#Make a scatter plot comparing the two slope patterns
scatter_plot = cpcrw.hvplot(kind='scatter', x='lidar_slope', y='is2_slope',
                            alpha=0.5)
scatter_plot.opts(xlabel='UAF slope', ylabel='ICESat-2 slope', fontscale=1.5)

Overall, it looks like the derived slopes agree pretty well with each other. Let's now see if there is any relationship between ICESat-2 slope and the snow depth residuals.

In [ ]:
# Make a scatter plot of ICESat-2 slope vs. IS2/UAF depth residuals
scatter_plot = cpcrw.hvplot(kind='scatter', x='is2_slope', y='snow_depth_residual',
                            alpha=0.5)
scatter_plot.opts(xlabel='ICESat-2 slope', ylabel='IS2-UAF depth residual [m]',
                  fontscale=1.5, ylim=(-1.5,1.5))